In [1]:
# IMPORT GENERIC PACKAGES
import numpy as np # numerical calc package
import pandas as pd # holds data
import matplotlib.pyplot as plt # plotting library
import seaborn as sns # pretty plotting
import pandas_profiling as pp
from xgboost import XGBClassifier

sns.set(rc={'figure.figsize':(20,10)})

from sklearn.model_selection import train_test_split # split dataset

from sklearn.linear_model import LogisticRegression # linear model for classification

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.datasets import make_hastie_10_2
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.datasets import load_digits
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import cross_val_score
from imblearn.over_sampling import SMOTE

In [2]:
data = pd.read_csv('census.csv')

In [3]:
#report=pp.ProfileReport(data)

In [4]:
#report.to_file('data_assessment.html')

In [5]:
data.head(5)

,age,workclass,education_level,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,Bachelors,13.0,Never-married,Adm-clerical,Not-in-family,White,Male,2174.0,0.0,40.0,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,13.0,United-States,<=50K
2,38,Private,HS-grad,9.0,Divorced,Handlers-cleaners,Not-in-family,White,Male,0.0,0.0,40.0,United-States,<=50K
3,53,Private,11th,7.0,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0.0,0.0,40.0,United-States,<=50K
4,28,Private,Bachelors,13.0,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0.0,0.0,40.0,Cuba,<=50K


In [6]:
data.columns

Index(['age', 'workclass', 'education_level', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
       'income'],
      dtype='object')

In [7]:
# write list of features we want to use
features = ['age', 'workclass', 'education_level', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country']

# Set X
X_temp = data[features]

# Convert text to numeric
X = pd.get_dummies(X_temp)

# Set y
y_temp = data["income"]
y_temp1 = pd.get_dummies(y_temp)

In [8]:
print(y_temp1)

       <=50K  >50K
0          1     0
1          1     0
2          1     0
3          1     0
4          1     0
...      ...   ...
45217      1     0
45218      1     0
45219      1     0
45220      1     0
45221      0     1

[45222 rows x 2 columns]


In [9]:
y=y_temp1.drop('>50K',axis=1)
print(y)

       <=50K
0          1
1          1
2          1
3          1
4          1
...      ...
45217      1
45218      1
45219      1
45220      1
45221      0

[45222 rows x 1 columns]


In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.4, random_state = 42)

In [11]:
# initialize model
gradient_booster = XGBClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0).fit(X_train, y_train)

# predictions
y_pred = gradient_booster.predict(X_test)

C:\Users\grachelquinones\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\grachelquinones\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [12]:
print(y_pred)

[1 1 1 ... 1 0 1]


In [13]:
target_names = ["<=50K", ">50K"]
print(classification_report(y_test,y_pred,target_names=target_names))

              precision    recall  f1-score   support

       <=50K       0.78      0.63      0.69      4453
        >50K       0.89      0.94      0.91     13636

    accuracy                           0.86     18089
   macro avg       0.83      0.78      0.80     18089
weighted avg       0.86      0.86      0.86     18089

